# Random Forest algorithm 
--- 

Random forest is a supercised learning algorithm. 

it has two variations:

    classification problem 
    regression problems
    
In our case, this algorithm can be used to rank the importance of variables in a regression of classification problem. With this we can detemine which anime is success.


## Import Libraries
---
Bottom are the essential libraries for random forest classification

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline


In [2]:
animelist_df = pd.read_csv('DataSets/Cleaned data/data_cleaned.csv')
animelist_df.shape


(9032, 77)

In [3]:
animelist_df.head()

,title,type,source,episodes,status,airing,rating,score,scored_by,rank,...,Related_Side story,Related_Alternative version,Related_Prequel,Related_Summary,Related_Other,Related_Spin-off,Related_Alternative setting,Related_Character,Related_Parent story,Related_Full story
0,Inu x Boku SS,TV,Manga,12,Finished Airing,False,PG-13 - Teens 13 or older,7.63,139250,1274.0,...,0,0,0,0,0,0,0,0,0,0
1,Seto no Hanayome,TV,Manga,26,Finished Airing,False,PG-13 - Teens 13 or older,7.89,91206,727.0,...,1,1,0,0,0,0,0,0,0,0
2,Shugo Chara!! Doki,TV,Manga,51,Finished Airing,False,PG - Children,7.55,37129,1508.0,...,0,0,1,0,0,0,0,0,0,0
3,Princess Tutu,TV,Original,38,Finished Airing,False,PG-13 - Teens 13 or older,8.21,36501,307.0,...,0,0,0,1,0,0,0,0,0,0
4,Bakuman. 3rd Season,TV,Manga,25,Finished Airing,False,PG-13 - Teens 13 or older,8.67,107767,50.0,...,0,0,2,0,1,0,0,0,0,0


In [4]:
animelist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9032 entries, 0 to 9031
Data columns (total 77 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   title                        9032 non-null   object 
 1   type                         9032 non-null   object 
 2   source                       9032 non-null   object 
 3   episodes                     9032 non-null   int64  
 4   status                       9032 non-null   object 
 5   airing                       9032 non-null   bool   
 6   rating                       9032 non-null   object 
 7   score                        9032 non-null   float64
 8   scored_by                    9032 non-null   int64  
 9   rank                         9032 non-null   float64
 10  popularity                   9032 non-null   int64  
 11  members                      9032 non-null   int64  
 12  favorites                    9032 non-null   int64  
 13  studio            

## Frequency distribution of values in variables
---

    Check the frequency counts of categorical variables

In [5]:
# count the genres in each category
cols = ['start_season','studio','type','source', 'rating']

for col in cols:
    print(animelist_df[col].value_counts())
    print()

start_season
Spring    2490
Fall      2225
Summer    2015
Winter    1951
0          351
Name: count, dtype: int64

studio
SmallStudio             2407
unknown                 2290
Toei Animation           487
Sunrise                  359
Madhouse                 274
J.C.Staff                269
Production I.G           214
Studio Deen              211
Studio Pierrot           187
TMS Entertainment        174
A-1 Pictures             155
OLM                      145
Nippon Animation         129
Xebec                    103
Gonzo                    103
Bones                     99
Shaft                     99
AIC                       92
Shin-Ei Animation         89
Kyoto Animation           89
Tatsunoko Production      89
Brain&#039;s Base         77
Silver Link.              71
Arms                      67
Satelight                 63
Production Reed           57
ufotable                  52
Doga Kobo                 51
Zexcs                     50
feel.                     49
Gainax  

In [6]:
# count the genres in each category
col_genres = [col for col in animelist_df.columns if 'Genre_' in col]
for col in col_genres:
    print(animelist_df[col].value_counts())   
    print()

Genre_Action
0    6490
1    2542
Name: count, dtype: int64

Genre_Adventure
0    7186
1    1846
Name: count, dtype: int64

Genre_Cars
0    8974
1      58
Name: count, dtype: int64

Genre_Comedy
0    5480
1    3552
Name: count, dtype: int64

Genre_Dementia
0    8923
1     109
Name: count, dtype: int64

Genre_Demons
0    8733
1     299
Name: count, dtype: int64

Genre_Drama
0    7385
1    1647
Name: count, dtype: int64

Genre_Ecchi
0    8420
1     612
Name: count, dtype: int64

Genre_Fantasy
0    7211
1    1821
Name: count, dtype: int64

Genre_Game
0    8830
1     202
Name: count, dtype: int64

Genre_Harem
0    8699
1     333
Name: count, dtype: int64

Genre_Hentai
0    7975
1    1057
Name: count, dtype: int64

Genre_Historical
0    8495
1     537
Name: count, dtype: int64

Genre_Horror
0    8706
1     326
Name: count, dtype: int64

Genre_Josei
0    8963
1      69
Name: count, dtype: int64

Genre_Kids
0    8484
1     548
Name: count, dtype: int64

Genre_Magic
0    8297
1     735
Name: co

In [7]:
# count the related in each category
col_related = [col for col in animelist_df.columns if 'Related_' in col]
for col in col_related:
    print(animelist_df[col].value_counts())   
    print()
    

Related_Adaptation
0     5174
1     3598
2      227
3       19
4        5
5        5
6        2
9        1
17       1
Name: count, dtype: int64

Related_Sequel
0    7032
1    1909
2      81
4       4
3       4
5       1
6       1
Name: count, dtype: int64

Related_Side story
0     8095
1      749
2      123
3       33
4       10
5        6
7        4
6        3
8        2
11       2
9        1
25       1
15       1
35       1
52       1
Name: count, dtype: int64

Related_Alternative version
0    8200
1     669
2     113
3      26
4      11
5       7
6       4
7       2
Name: count, dtype: int64

Related_Prequel
0    7249
1    1707
2      64
3       9
6       1
5       1
4       1
Name: count, dtype: int64

Related_Summary
0    8686
1     305
2      32
3       7
4       1
6       1
Name: count, dtype: int64

Related_Other
0     7754
1      956
2      184
3       86
5       18
4       15
6        8
7        4
8        4
10       2
12       1
Name: count, dtype: int64

Related_Spin-off
0 

In [8]:
col_s = pd.concat([cols, col_genres, col_related], axis = 1)
for col in col_s:
    print(animelist_df[col].value_counts())   
    print()

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

### Explore "Season" Variable
---

In [ ]:
animelist_df['start_season'].value_counts()

In [ ]:
for col in animelist_df.select_dtypes(include=['object']).columns:
    print(col)
    print(animelist_df[col].unique())
    print()


In [ ]:
X = animelist_df.drop(['start_season'], axis=1)

y = animelist_df['start_season']


In [ ]:
# split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

# check the shape of X_train and X_test
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# check data types in X_train

X_train.dtypes

In [ ]:
X_train.head()

In [ ]:
# Initialize an empty list to store the names of object columns
object_columns = []

# Iterate through each column
for col in animelist_df.columns:
    # Check if the column is of type 'object'
    if animelist_df[col].dtype == 'object':
         # Print the column name and its unique values
        print(col)
        print(animelist_df[col].unique())
        print()
        # Save the column name to the list of object columns
        object_columns.append(col)

print(cols)

In [ ]:
import category_encoders as ce

# Encode categorical variables with ordinal encoding
encoder = ce.OrdinalEncoder(cols=['type', 'source', 'rating', 'studio', 'start_season'])


X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)


## Random Forest Classifier model with default parameters
---



In [ ]:
# import Random Forest classifier
from sklearn.ensemble import RandomForestClassifier

# instantiate the classifier 
rfc = RandomForestClassifier(random_state=0)

#X_train = X_train.drop(columns=['title'])

# fit the model
rfc.fit(X_train, y_train)


# Predict the Test set results
y_pred = rfc.predict(X_test)



# Check accuracy score 
from sklearn.metrics import accuracy_score

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))